## ひろしまQuest2020#stayhome【球種予測部門】

In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import gc
%matplotlib inline
pd.set_option('display.max_Columns', 100)

In [36]:
ALL_MERGE = 'intermediate/all_merge_9.f'
SUBMIT = 'submit/lgbm_9.csv'

In [37]:
all_pitch = pd.read_feather(ALL_MERGE)
all_pitch.shape

(778767, 172)

### 不要な列を削除

In [38]:
all_pitch.drop(
    columns=[
        '年度', 
        '試合ID', 
        'ホームチームID', 'アウェイチームID', 
        '投手ID', '投手チームID', 
        '打者ID', '打者チームID', 
        'プレイ前走者状況', 
        '捕手ID', 
        'opening_date', 'game_date',
        'start_time', 'game_time', 'elapsed_time'
    ], inplace=True)

### Rename

In [39]:
all_pitch.rename(columns={
    'データ内連番': 'No',
    '投球位置区域': 'pitch_area',
    '試合内連番': 'No_in_game',
    '試合内投球数': 'pitch_cnt_in_game',
    'イニング': 'inning',
    'イニング内打席数': 'bat_cnt_in_inning',
    '打席内投球数': 'pitch_cnt_in_bat',
    '投手登板順': 'pitch_order',
    '投手試合内対戦打者数': 'player_cnt_in_game',
    '投手試合内投球数': 'pitcher_cnt_in_game',
    '投手イニング内投球数': 'pitcher_cnt_in_inning',
    '打者打順': 'bat_order',
    '打者試合内打席数': 'bat_cnt_in_game',
    'プレイ前ホームチーム得点数': 'home_point',
    'プレイ前アウェイチーム得点数': 'away_point',
    'プレイ前アウト数': 'out_cnt',
    'プレイ前ボール数': 'ball_cnt',
    'プレイ前ストライク数': 'strike_cnt',
}, inplace=True)

In [40]:
print(all_pitch.shape)
column_cnt = len(all_pitch.columns)
all_pitch.head()

(778767, 157)


,No,ball,pitch_area,No_in_game,pitch_cnt_in_game,inning,bat_cnt_in_inning,pitch_cnt_in_bat,pitch_order,player_cnt_in_game,pitcher_cnt_in_game,pitcher_cnt_in_inning,bat_order,bat_cnt_in_game,home_point,away_point,out_cnt,ball_cnt,strike_cnt,first,second,third,base_cnt,pitch_LR,bat_LR,top_bot,role,pos_pit,date_from_opening,elapsed_min,nighter,ce-pa,league,home,point_diff,runner_23,bant,squize,cleanup,pinch,fourball,savepoint,ball_cnt_0-0,ball_cnt_0-1,ball_cnt_0-2,ball_cnt_0-3,ball_cnt_1-0,ball_cnt_1-1,ball_cnt_1-2,ball_cnt_1-3,...,firm,height,weight,draft_order,salary,foreigner,c_straight,c_curve,c_slider,c_shoot,c_fork,c_changeup,c_sinker,c_cutball,c_total,cat_game_cnt,cat_inning_cnt,cat_batter_cnt,cat_inning_per_game,cat_batter_per_game,cat_ball_per_game,cat_batter_per_inning,cat_ball_per_inning,cat_ball_per_batter,company,univ,highsch,age,play_year,salary_year,salary_x_year,bmi,salary_dif_p-b,play_year_dif_p-b,age_dif_p-b,salary_year_dif_p-b,salary_x_year_dif_p-b,bmi_dif_p-b,salary_dif_p-c,play_year_dif_p-c,age_dif_p-c,salary_year_dif_p-c,salary_x_year_dif_p-c,bmi_dif_p-c,salary_dif_b-c,play_year_dif_b-c,age_dif_b-c,salary_year_dif_b-c,salary_x_year_dif_b-c,bmi_dif_b-c
0,0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0.0,1.0,1,1,0,0,0.000000,1,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,...,0,178,83,1.0,5000,0,0.419137,0.072469,0.23547,0.105491,0.052524,0.050307,0.000886,0.063715,18049.0,137.0,1114.0,4625.0,8.131387,33.759124,131.744526,4.151706,16.201975,3.902486,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725
1,1,0.0,8.0,2,2,1,1,2,1,1,2,2,1,1,0,0,0,0,1,0,0,0,0,0.0,1.0,1,1,0,0,0.200000,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,...,0,178,83,1.0,5000,0,0.419137,0.072469,0.23547,0.105491,0.052524,0.050307,0.000886,0.063715,18049.0,137.0,1114.0,4625.0,8.131387,33.759124,131.744526,4.151706,16.201975,3.902486,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725
2,2,0.0,5.0,3,3,1,1,3,1,1,3,3,1,1,0,0,0,0,2,0,0,0,0,0.0,1.0,1,1,0,0,0.383333,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,178,83,1.0,5000,0,0.419137,0.072469,0.23547,0.105491,0.052524,0.050307,0.000886,0.063715,18049.0,137.0,1114.0,4625.0,8.131387,33.759124,131.744526,4.151706,16.201975,3.902486,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725
3,3,0.0,12.0,4,4,1,2,1,1,2,4,4,2,1,0,0,1,0,0,0,0,0,0,0.0,0.0,1,1,0,0,1.016667,1,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,...,0,178,83,1.0,5000,0,0.419137,0.072469,0.23547,0.105491,0.052524,0.050307,0.000886,0.063715,18049.0,137.0,1114.0,4625.0,8.131387,33.759124,131.744526,4.151706,16.201975,3.902486,1,1,0,28,4.0,1250.0,20000.0,26.196187,16960,-16.0,-11,3680.0,-10880.0,3.191314,19000,2.0,1,2750.0,124000.0,-0.165367,2040,18.0,12,-930.0,134880.0,-3.356681
4,4,0.0,8.0,5,5,1,2,2,1,2,5,5,2,1,0,0,1,0,1,0,0,0,0,0.0,0.0,1,1,0,0,1.200000,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,...,0,178,83,1.0,5000,0,0.419137,0.072469,0.23547,0.105491,0.052524,0.050307,0.000886,0.063715,18049.0,137.0,1114.0,4625.0,8.131387,33.759124,131.744526,4.151706,16.201975,3.902486,1,1,0,28,4.0,1250.0,20000.0,26.196187,16960,-16.0,-11,3680.0,-10880.0,3.191314,19000,2.0,1,2750.0,124000.0,-0.165367,2040,18.0,12,-930.0,134880.0,-3.356681


### train
- 行数: 257117

In [41]:
train = all_pitch.dropna(subset=['pitch_area'])
train.shape

(257117, 157)

### test
- 行数: 521650

In [42]:
test = all_pitch[all_pitch['pitch_area'].isnull()]
test.shape

(521650, 157)

In [43]:
del all_pitch
gc.collect()

70

In [44]:
train_d = train.drop([
    'No', 
    'pitch_area', 
    'ball'
], axis=1)

In [45]:
test_d = test.drop([
    'No', 
    'pitch_area', 
    'ball'
], axis=1)

## Dataset作成

In [46]:
lgb_train = lgb.Dataset(train_d, train['ball'])

### LGBM学習
- multiclass : クラス数=8
- tuning結果のパラメータを使用
- 'learning_rate' : 0.1

In [47]:
%%time
lgb_param = {
        'objective' : 'multiclass',
        'boosting_type': 'gbdt',
        'metric' : 'multi_logloss',
        'num_class' : 8,
        'seed' : 0,
        'learning_rate' : 0.1,
        'lambda_l1': 6.9923570049658075, 
        'lambda_l2': 0.002378623984798833, 
        'num_leaves': 18, 
        'feature_fraction': 0.45199999999999996, 
        'bagging_fraction': 0.9799724836460725, 
        'bagging_freq': 4, 
        'min_child_samples': 20
    }

# 学習
cv_results = lgb.cv(lgb_param, lgb_train,
                    num_boost_round=15000,
                    early_stopping_rounds=100,
                    verbose_eval=100,
                    nfold=4)

num_boost_round = len(cv_results['multi_logloss-mean'])
print('Best num_boost_round:', num_boost_round)
best_cv_score = cv_results['multi_logloss-mean'][-1]
print('Best CV score:', best_cv_score)

[100]	cv_agg's multi_logloss: 1.15182 + 0.00217455
[200]	cv_agg's multi_logloss: 1.13045 + 0.00196231
[300]	cv_agg's multi_logloss: 1.12309 + 0.0020824
[400]	cv_agg's multi_logloss: 1.11857 + 0.00206845
[500]	cv_agg's multi_logloss: 1.1155 + 0.00221064
[600]	cv_agg's multi_logloss: 1.11332 + 0.00210684
[700]	cv_agg's multi_logloss: 1.11175 + 0.00229951
[800]	cv_agg's multi_logloss: 1.11067 + 0.00226031
[900]	cv_agg's multi_logloss: 1.10993 + 0.00231447
[1000]	cv_agg's multi_logloss: 1.10958 + 0.00239318
[1100]	cv_agg's multi_logloss: 1.10937 + 0.00240076
[1200]	cv_agg's multi_logloss: 1.10938 + 0.00251967
Best num_boost_round: 1100
Best CV score: 1.1093728644236824
CPU times: user 1h 20min 4s, sys: 5.59 s, total: 1h 20min 9s
Wall time: 10min 4s


In [48]:
best_iter = int(num_boost_round * 1.1)

### 全体で再学習

In [49]:
%%time
lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)

CPU times: user 24min 51s, sys: 1.06 s, total: 24min 52s
Wall time: 3min 6s


### Feature Importance

In [50]:
fi = lgb_model.feature_importance()
fn = lgb_model.feature_name()
df_feature_importance = pd.DataFrame({'feat_name':fn, 'feat_imp':fi})
df_feature_importance.sort_values('feat_imp', inplace=True)
df_feature_importance.tail(30)

,feat_name,feat_imp
15,strike_cnt,1878
138,age_dif_p-b,1894
64,fork_pit,1938
88,height_bat,1983
14,ball_cnt,2012
89,weight_bat,2030
136,salary_dif_p-b,2134
60,straight_pit,2144
94,bat_game_cnt_bat,2154
140,salary_x_year_dif_p-b,2155


In [51]:
%%time
predict = lgb_model.predict(test_d, num_iteration = lgb_model.best_iteration)

CPU times: user 6min 33s, sys: 1.67 s, total: 6min 34s
Wall time: 51.8 s


## 結果出力

In [52]:
submit = pd.DataFrame(predict)
submit.reset_index(inplace=True)
print(submit.shape)
submit.head()

(521650, 9)


,index,0,1,2,3,4,5,6,7
0,0,0.307205,0.029614,0.135185,0.372699,0.002732,0.000017,0.000023,0.152525
1,1,0.302134,0.078794,0.153014,0.398776,0.003755,0.000023,0.000015,0.063491
2,2,0.310473,0.088823,0.147199,0.305373,0.069813,0.000075,0.000030,0.078214
3,3,0.284600,0.119064,0.208629,0.288032,0.020850,0.000057,0.000023,0.078744
4,4,0.212548,0.050627,0.319085,0.309034,0.014596,0.000056,0.000021,0.094033


### 2017の平均
- ストレート           0.468254
- カーブ             0.074725
- スライダー           0.185806
- シュート            0.070633
- フォーク            0.083013
- チェンジアップ         0.054217
- シンカー            0.011361
- カットボール          0.051992

In [53]:
submit.mean()

index    260824.500000
0             0.557966
1             0.062042
2             0.174119
3             0.046998
4             0.072082
5             0.037540
6             0.007152
7             0.042100
dtype: float64

In [54]:
submit.to_csv(SUBMIT, header=False, index=False)

### SIGNATE

In [55]:
'signate submit --competition-id=275 ~/work/signate/baseball/{} --note feat={}_cv={}'.format(SUBMIT, column_cnt, best_cv_score)

'signate submit --competition-id=275 ~/work/signate/baseball/submit/lgbm_8.csv --note feature=157'

### 評価結果
#### 8 (157)
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1100
- Best CV score: 1.1093728644236824
- Wall time: 10min 4s
- 評価結果  : 1.4925832

#### 7 (235)
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1204
- Best CV score: 1.108590082437519
- Wall time: 12min 56s
- 評価結果  : 1.4946937

#### 6 (207)
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- Best num_boost_round: 1135
- Best CV score: 1.1086837935965734
- Wall time: 11min 54s
- 評価結果  : 1.3999194

#### 5-tune (参考)
- 'num_leaves' : 18,
- 'learning_rate' : 0.05,
- Best num_boost_round: 2395
- Best CV score: 1.1103399384119979
- Wall time: 18min 29s (n1-standard-8)

#### 5-tune (135)
- 'num_leaves' : 18,
- 'learning_rate' : 0.1,
- CV score: 1.1112402566616455
- 評価結果  : 1.3991780
- CV Wall time: 27min 48s (local) -> 18min 24s (n1-standard-4) -> 9min 3s (n1-standard-8)

#### 5 (135)
- 'num_leaves' : 31,
- 'learning_rate' : 0.1,
- CV score: 1.1161825304353852
- 評価結果  : 1.4252286

#### 4 (128)
- CV score: 1.1158111693845418
- 評価結果  : 1.4299265

#### 3 (97)
- CV score: 1.1172290313076036
- 評価結果  : 1.4335806